In [14]:
#$ RELATIONAL DATABASES:

### 1) INTRODUCTION 

    ## What is a realtional database ? Example of Company's order book database (Order Table, Customer table and employee table)

        # based on realtional model of data. first described by "Tedd" Codd in late 1960s
        # Example we have Database consisting of many tables (like Order Table, customer table and employees table) 
        # These tables are analogous to  dataframes (with unqiue keys (to access rows, ex: Order Id in order table) and there attributes.
        # Tables in realtional databases are unique because they are linked.
        # For example : via the order id key we can access the Order Table, which also has Customer_id and employee_id information
        # .. which can be used as keys to access Cusomter Table and Employee table for any further information (called query)
        # So given an order ID, we can immediate look up the relevant customer and employee information, 
        # .. this means we dont need to store all the customer and employee inforamtion (ex there names etc) 
        # .. information in the order table, we can just pass a query to look it up in relational databases.
    
    ## Relational Model used in Relational databases is summarized in Codd's 12 rules/ commandments 
        # Consist of 13 rules - as first rule is indexed 0. haha..
        # These rules describe what the relational database management system should adhere to to be considered relational
        
    ## Common Relational Database Management Systems all of which use Sequal query language  
        # PostgresSQL
        # MySQL
        # SQLite
        # .... where SQL is acronyn for structured Query Language which describes how to communicate with a database to access and update it.

### 2) Basic Querying : QUerying individual tables of relational databases 

    # What we need is to get out data from a database using SQL that is structured query language
    # There are several way : 
        
        # Method 1 : Workflow of such a process via data base engine creation and opening connections 
            
            # Step 1 : Import packages and functions
            # Step 2 : Create the data base engine and look at the tables contained
            # Step 3 : Connect to the engine 
            # Step 4 : Query the database (getting all or specific columns of specific tables and ordering the SQL records via ORDER BY)
            # Step 5 : Save results of the query to the dataframe
            # Step 6 : Close the connection
        
         # Method 2 : Work flow of alternative process via context manager to open connection
            
            #  open the connection via context manager, rest same as Method 1 to do query process

         # Method 3 :  Querying the relational databases directly with pandas
            
            # converts the 4 line of code required to query the database by context manager to store as dataframe to just one line
            # After creating engine (Step 1 -3 of Method 1)
            # 1 line code for querying using pandas : df = pd.read_sql_query("SQL QUery to execute", engine)
            ## .... first argument is the SQL query we need to execute
            ## ... second argument is the engine we want to connect to
            
### 3) Advanced Querying : Exploiting Linked Table Relationships - the real usecase of relational databases

        # Example querying multiple tables via their keys which correspond to coloumns in originally accessed data table
        # Example get the order ID to access Orders Table 
        # .. and then use Customer ID and Employee ID from it as keys to access Customer and Employee table information 
        # JOINing the queried data -- example of INNER JOIN here
        
#### DIFFERENT SQL QUERIES COVERED HERE IN EXAMPLES : 
                # SELECT
                # WHERE
                # JOIN 
     


In [19]:
#  Method 1 : Workflow of such a generic process of querying SQL data 

# Step 1,2) :  Importing fucntion from packages, creating data base engine and looking at table names
    
            # SQLite Database as example
            # Many packages to access SQLite databases such as:  SQLite3 and SQLAlchemy 
            # SQLAlchemy will be used as it works with many other relational database management systems such as PostgresSQL and MySQL
            # using create_engine() function from sqlalchemy package to fire up a SQL engine that communicates our queries to database
            
from sqlalchemy import create_engine  # <-- Step 1 - importing pandas and create_engine() from sqlalchemy package

import pandas as pd
            
engine = create_engine('sqlite:///Chinook.sqlite') # <--  Step 2a : creates engine and accepts only one argument 
                                                   # <-- string containing database type, and name of database in working directory

table_names = engine.table_names()  # <-- Step 2b - Find the table names contianed by the connected database : 
                                     #.. using table_names() method on engine1

print("Table names are : " + str(table_names)) #<-- # Print the table names     


Table names are : ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [20]:
  # Step 3 : Connecting to engine  : apply method .connect() on engine

con = engine.connect() # <-- creates a connection object
    

In [21]:
 # Step 4, 5) Querying the connected relational database, 
    
    # Example Query : select all columns from required table, renaming coloumns appropriately and saving as dataframe
    
    # Querying the relational database connected by using SQL queries
            # Basic SQL Query - select * Table_Name  <--- where table name is the name of any of the tables in the database
                                                 # <-- this query returns all of the coloumns of all of the rows
            # In order to use such  SQL Query in python we use  sqlalchemy and pandas.
           
        #  (i)  To execute a sql query in python use method .execute("SQL Query") on connected engine con : con.execute()
            
rs = con.execute('SELECT * FROM Album')  # <-- pass the relevant SQL Query as the argument to the method .excute()
                                         # <-- here the SQL query as argument is to select all coloumns of all rows in table album
                                         # <-- results in a sqlalchemy results object with coloumn names as keys
        
        # (ii) To convert the resulting sqlalchemy results object to dataframe: apply method .fetchall() to results object
        
df = pd.DataFrame(rs.fetchall())  # <-- fetchall() fetches all rows as ecpected

print(df.head()) # <-- print the head of the dataframe. Rows looks fine, but coloumn names need to be redefined

        # (iii) Rename the coloumn names of output dataframe df to output of method .key() keys on the results object rs

df.columns = rs.keys() # <-- renames the coloumn names in df
print(df.head())

  # Step 6) close the connection : use method close() on connection con
    
con.close() #<-- closes the connection


   0                                      1  2
0  1  For Those About To Rock We Salute You  1
1  2                      Balls to the Wall  2
2  3                      Restless and Wild  2
3  4                      Let There Be Rock  1
4  5                               Big Ones  3
   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3


In [25]:
# Method 2 : Work flow of alternative process via context manager to open connection

    # Example Query: To query LastName and Title coloumn of Employee table, and get the first 5 rows only
    
from sqlalchemy import create_engine  

import pandas as pd
            
engine = create_engine('sqlite:///Chinook.sqlite')

with engine.connect() as con :  # <-- open a connection named con to the engine using method .connect() -- context manager
    rs = con.execute("SELECT LastName, LastName FROM Employee") # <-- selects LastName and Title from Employee Table
    df = pd.DataFrame(rs.fetchmany(size=5))  # <-- only stores the first 5 rows value as dataframe from the sqlalchemy result object
    df.columns = rs.keys() # <-- renaming the columns in df using keys of rs tht is column names
    print(df)           


  LastName LastName
0    Adams    Adams
1  Edwards  Edwards
2  Peacock  Peacock
3     Park     Park
4  Johnson  Johnson


In [27]:
        # Example Query : Select all records from the Employee table where 'EmployeeId' >= 6. 
                     
# Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

# Open engine in context manager
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Employee where EmployeeID >= 6")  # <-- required sql query 
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

# Print the head of the DataFrame df
print(df.head())
    

   EmployeeId  LastName FirstName       Title  ReportsTo            BirthDate  \
0           6  Mitchell   Michael  IT Manager          1  1973-07-01 00:00:00   
1           7      King    Robert    IT Staff          6  1970-05-29 00:00:00   
2           8  Callahan     Laura    IT Staff          6  1968-01-09 00:00:00   

              HireDate                      Address        City State Country  \
0  2003-10-17 00:00:00         5827 Bowness Road NW     Calgary    AB  Canada   
1  2004-01-02 00:00:00  590 Columbia Boulevard West  Lethbridge    AB  Canada   
2  2004-03-04 00:00:00                  923 7 ST NW  Lethbridge    AB  Canada   

  PostalCode              Phone                Fax                    Email  
0    T3B 0C5  +1 (403) 246-9887  +1 (403) 246-9899  michael@chinookcorp.com  
1    T1K 5N8  +1 (403) 456-9986  +1 (403) 456-8485   robert@chinookcorp.com  
2    T1H 1Y8  +1 (403) 467-3351  +1 (403) 467-8772    laura@chinookcorp.com  


In [29]:
# Example Query : Ordering your SQL records with ORDER BY
    
        #selects all records from the Employee table and orders them in increasing order by the column BirthDate 
        
    # Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

# Open engine in context manager
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Employee ORDER BY BirthDate")
    df = pd.DataFrame(rs.fetchall())

    # Set the DataFrame's column names
    df.columns = rs.keys()

# Print head of DataFrame
print(df.head())

   EmployeeId  LastName FirstName                Title  ReportsTo  \
0           4      Park  Margaret  Sales Support Agent        2.0   
1           2   Edwards     Nancy        Sales Manager        1.0   
2           1     Adams    Andrew      General Manager        NaN   
3           5   Johnson     Steve  Sales Support Agent        2.0   
4           8  Callahan     Laura             IT Staff        6.0   

             BirthDate             HireDate              Address        City  \
0  1947-09-19 00:00:00  2003-05-03 00:00:00     683 10 Street SW     Calgary   
1  1958-12-08 00:00:00  2002-05-01 00:00:00         825 8 Ave SW     Calgary   
2  1962-02-18 00:00:00  2002-08-14 00:00:00  11120 Jasper Ave NW    Edmonton   
3  1965-03-03 00:00:00  2003-10-17 00:00:00         7727B 41 Ave     Calgary   
4  1968-01-09 00:00:00  2004-03-04 00:00:00          923 7 ST NW  Lethbridge   

  State Country PostalCode              Phone                Fax  \
0    AB  Canada    T2P 5G3  +1 (403)

In [31]:
        # Method 3 :  Querying the relational databases directly with pandas
    
            # After creating engine (Step 1 -3 of Method 1) no need of context manager as in Method 2.
            # 1 line code for querying the table using pandas including renaming the table column names :

# Example Query : select all records from the Employee table where the EmployeeId is greater than or equal to 6 
                    # .. and ordered by BirthDate 

from sqlalchemy import create_engine  
import pandas as pd            
engine = create_engine('sqlite:///Chinook.sqlite')
 
df = pd.read_sql_query("SELECT * FROM Employee where EmployeeID >= 6 ORDER BY BirthDate", engine) # <-- one line query using pandas

# <-- equivalent to these 4 lines
    #   with engine.connect() as con:
        #    rs = con.execute("SELECT * FROM Employee ORDER BY BirthDate")
        #    df = pd.DataFrame(rs.fetchall())
        #    df.columns = rs.keys()

print(df.head()) 

   EmployeeId  LastName FirstName       Title  ReportsTo            BirthDate  \
0           8  Callahan     Laura    IT Staff          6  1968-01-09 00:00:00   
1           7      King    Robert    IT Staff          6  1970-05-29 00:00:00   
2           6  Mitchell   Michael  IT Manager          1  1973-07-01 00:00:00   

              HireDate                      Address        City State Country  \
0  2004-03-04 00:00:00                  923 7 ST NW  Lethbridge    AB  Canada   
1  2004-01-02 00:00:00  590 Columbia Boulevard West  Lethbridge    AB  Canada   
2  2003-10-17 00:00:00         5827 Bowness Road NW     Calgary    AB  Canada   

  PostalCode              Phone                Fax                    Email  
0    T1H 1Y8  +1 (403) 467-3351  +1 (403) 467-8772    laura@chinookcorp.com  
1    T1K 5N8  +1 (403) 456-9986  +1 (403) 456-8485   robert@chinookcorp.com  
2    T3B 0C5  +1 (403) 246-9887  +1 (403) 246-9899  michael@chinookcorp.com  


In [32]:
### 3) Advanced Querying : Exploiting Linked Table Relationships - the real usecase of relational databases

        # table_name.column_name notation is to select the specified column of specified table 
        # selecting desired coloumns from tableA INNER JOIN Tableb 
        # .. and stating the link between them tableA.matchin_column = tableB.matching_column
        
# Example Query : - perform your first INNER JOIN
                # - For each record in the Album table, you'll extract the Title along with the Name of the Artist.
                # - The latter will come from the Artist table 
                # - so you will need to INNER JOIN these two tables on the ArtistID column of both.

from sqlalchemy import create_engine  
import pandas as pd            
engine = create_engine('sqlite:///Chinook.sqlite')            

df = pd.read_sql_query("SELECT Title, Name FROM Album INNER JOIN Artist on Album.ArtistID = Artist.ArtistID", engine)

# Print head of DataFrame df
print(df.head())

                                   Title       Name
0  For Those About To Rock We Salute You      AC/DC
1                      Balls to the Wall     Accept
2                      Restless and Wild     Accept
3                      Let There Be Rock      AC/DC
4                               Big Ones  Aerosmith


In [33]:
# Example Query - select all records from PlaylistTrack INNER JOIN Track on PlaylistTrack.TrackId = Track.TrackId 
                # .. that satisfy the condition Milliseconds < 250000
    
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Chinook.sqlite')
df = pd.read_sql_query("SELECT * FROM PlaylistTrack INNER JOIN Track on PlaylistTrack.TrackId = Track.TrackId where Milliseconds < 250000", engine)
print(df.head())

   PlaylistId  TrackId  TrackId              Name  AlbumId  MediaTypeId  \
0           1     3390     3390  One and the Same      271            2   
1           1     3392     3392     Until We Fall      271            2   
2           1     3393     3393     Original Fire      271            2   
3           1     3394     3394       Broken City      271            2   
4           1     3395     3395          Somedays      271            2   

   GenreId Composer  Milliseconds    Bytes  UnitPrice  
0       23     None        217732  3559040       0.99  
1       23     None        230758  3766605       0.99  
2       23     None        218916  3577821       0.99  
3       23     None        228366  3728955       0.99  
4       23     None        213831  3497176       0.99  
